In [ ]:
!pip install openai anthropic cohere stripe supabase firebase-admin azure-identity azure-mgmt-core requests python-dotenv notion-client spotipy tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import requests
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

api_keys = {
    "Key": "Enter all keys one by one" 
}

def check_openai_key(api_key):
    """Check OpenAI API key validity"""
    try:
        import openai
        client = openai.OpenAI(api_key=api_key)
        response = client.models.list()
        return True, "Valid - Models retrieved successfully"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_anthropic_key(api_key):
    """Check Anthropic API key validity"""
    try:
        import anthropic
        client = anthropic.Anthropic(api_key=api_key)
        # Try a simple completion
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=5,
            messages=[{"role": "user", "content": "Hi"}]
        )
        return True, "Valid - Message created successfully"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_cohere_key(api_key):
    """Check Cohere API key validity"""
    try:
        import cohere
        co = cohere.Client(api_key)
        response = co.generate(
            model='command',
            prompt='Hello',
            max_tokens=5
        )
        return True, "Valid - Generation successful"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_stripe_keys(secret_key, publishable_key):
    """Check Stripe API keys validity"""
    try:
        import stripe
        stripe.api_key = secret_key
        # Test the secret key
        stripe.Account.retrieve()
        return True, "Valid - Account retrieved successfully"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_supabase_keys(url, anon_key, service_key):
    """Check Supabase credentials"""
    try:
        from supabase import create_client, Client
        supabase = create_client(url, anon_key)
        # Try to get user (will fail if not authenticated but connection works)
        result = supabase.auth.get_user()
        return True, "Valid - Connection established"
    except Exception as e:
        # Check if it's just auth issue vs connection issue
        if "Invalid JWT" in str(e) or "JWT" in str(e):
            return True, "Valid - Connection works (auth expected to fail)"
        return False, f"Invalid - {str(e)}"

def check_github_token(token):
    """Check GitHub token validity"""
    try:
        headers = {'Authorization': f'token {token}'}
        response = requests.get('https://api.github.com/user', headers=headers)
        if response.status_code == 200:
            return True, "Valid - User info retrieved"
        else:
            return False, f"Invalid - Status code: {response.status_code}"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_spotify_credentials(client_id, client_secret):
    """Check Spotify API credentials"""
    try:
        import spotipy
        from spotipy.oauth2 import SpotifyClientCredentials

        client_credentials_manager = SpotifyClientCredentials(
            client_id=client_id,
            client_secret=client_secret
        )
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        # Try a simple search
        results = sp.search(q='test', limit=1)
        return True, "Valid - Search successful"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_twitter_credentials(api_key, api_secret, access_token, access_token_secret):
    """Check Twitter API credentials"""
    try:
        import tweepy

        auth = tweepy.OAuthHandler(api_key, api_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)

        # Try to get user info
        api.verify_credentials()
        return True, "Valid - Credentials verified"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_notion_credentials(api_key, database_id):
    """Check Notion API credentials"""
    try:
        from notion_client import Client

        notion = Client(auth=api_key)
        # Try to retrieve database info
        response = notion.databases.retrieve(database_id=database_id)
        return True, "Valid - Database retrieved"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_firebase_config(api_key, project_id):
    """Check Firebase configuration"""
    try:
        # Simple validation of Firebase config format
        if not api_key or not project_id:
            return False, "Invalid - Missing required fields"

        # Try to validate with Firebase REST API
        url = f"https://identitytoolkit.googleapis.com/v1/accounts:signUp?key={api_key}"
        response = requests.post(url, json={})

        # Even if signup fails, if we get a proper error response, the key format is valid
        if response.status_code in [400, 403]:
            return True, "Valid - API key format correct"

        return False, f"Invalid - Status code: {response.status_code}"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

def check_azure_credentials(client_id, client_secret, tenant_id):
    """Check Azure credentials"""
    try:
        from azure.identity import ClientSecretCredential

        credential = ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )

        # Try to get a token
        token = credential.get_token("https://management.azure.com/.default")
        return True, "Valid - Token acquired successfully"
    except Exception as e:
        return False, f"Invalid - {str(e)}"

# Main validation function
def validate_all_credentials():
    print("🔍 COMPREHENSIVE API KEY VALIDATION")
    print("=" * 50)
    print(f"Validation started at: {datetime.now()}")
    print()

    results = {}

    # OpenAI
    print("🤖 Checking OpenAI API Key...")
    valid, msg = check_openai_key(api_keys['OPENAI_API_KEY'])
    results['OpenAI'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Anthropic
    print("\n🧠 Checking Anthropic API Key...")
    valid, msg = check_anthropic_key(api_keys['ANTHROPIC_API_KEY'])
    results['Anthropic'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Cohere
    print("\n🔤 Checking Cohere API Key...")
    valid, msg = check_cohere_key(api_keys['COHERE_API_KEY'])
    results['Cohere'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Stripe
    print("\n💳 Checking Stripe Keys...")
    valid, msg = check_stripe_keys(api_keys['STRIPE_SECRET_KEY'], api_keys['STRIPE_PUBLISHABLE_KEY'])
    results['Stripe'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Supabase
    print("\n🗃️ Checking Supabase Credentials...")
    valid, msg = check_supabase_keys(
        api_keys['SUPABASE_URL'],
        api_keys['SUPABASE_ANON_KEY'],
        api_keys['SUPABASE_SERVICE_ROLE_KEY']
    )
    results['Supabase'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # GitHub
    print("\n🐙 Checking GitHub Token...")
    valid, msg = check_github_token(api_keys['GITHUB_TOKEN'])
    results['GitHub'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Spotify
    print("\n🎵 Checking Spotify Credentials...")
    valid, msg = check_spotify_credentials(
        api_keys['SPOTIFY_CLIENT_ID'],
        api_keys['SPOTIFY_CLIENT_SECRET']
    )
    results['Spotify'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Twitter
    print("\n🐦 Checking Twitter Credentials...")
    valid, msg = check_twitter_credentials(
        api_keys['TWITTER_API_KEY'],
        api_keys['TWITTER_API_SECRET'],
        api_keys['TWITTER_ACCESS_TOKEN'],
        api_keys['TWITTER_ACCESS_TOKEN_SECRET']
    )
    results['Twitter'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Notion
    print("\n📝 Checking Notion Credentials...")
    valid, msg = check_notion_credentials(
        api_keys['NOTION_API_KEY'],
        api_keys['NOTION_DATABASE_ID']
    )
    results['Notion'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Firebase
    print("\n🔥 Checking Firebase Configuration...")
    valid, msg = check_firebase_config(
        api_keys['FIREBASE_API_KEY'],
        api_keys['FIREBASE_PROJECT_ID']
    )
    results['Firebase'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Azure
    print("\n☁️ Checking Azure Credentials...")
    valid, msg = check_azure_credentials(
        api_keys['AZURE_CLIENT_ID'],
        api_keys['AZURE_CLIENT_SECRET'],
        api_keys['AZURE_TENANT_ID']
    )
    results['Azure'] = {'valid': valid, 'message': msg}
    print(f"   {'✅' if valid else '❌'} {msg}")

    # Summary
    print("\n" + "=" * 50)
    print("📊 VALIDATION SUMMARY")
    print("=" * 50)

    valid_count = sum(1 for r in results.values() if r['valid'])
    total_count = len(results)

    print(f"✅ Valid: {valid_count}/{total_count}")
    print(f"❌ Invalid: {total_count - valid_count}/{total_count}")
    print(f"Success Rate: {(valid_count/total_count)*100:.1f}%")

    print("\n🔍 DETAILED RESULTS:")
    for service, result in results.items():
        status = "✅ VALID" if result['valid'] else "❌ INVALID"
        print(f"  {service}: {status}")
        print(f"    └── {result['message']}")

    print(f"\nValidation completed at: {datetime.now()}")

    return results

# Run the validation
validation_results = validate_all_credentials()

🔍 COMPREHENSIVE API KEY VALIDATION
Validation started at: 2025-05-23 20:16:44.675527

🤖 Checking OpenAI API Key...
   ❌ Invalid - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-live-********************************************z4ZK. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

🧠 Checking Anthropic API Key...
   ❌ Invalid - Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🔤 Checking Cohere API Key...
   ❌ Invalid - status_code: 401, body: {'message': 'invalid api token'}

💳 Checking Stripe Keys...
   ❌ Invalid - Invalid API Key provided: sk_live_*************************************************************************************************Zz2L

🗃️ Checking Supabase Credentials...
   ✅ Valid - Connection established

🐙 Checking GitHub Token...
   ❌ Invalid - Status code: 401

🎵 Checking Spotif

   ❌ Invalid - Authentication failed: Unable to get authority configuration for https://login.microsoftonline.com/91bc4f96-3a3b-4a8b-9f6f-552ddf0fcd81. Authority would typically be in a format of https://login.microsoftonline.com/your_tenant or https://tenant_name.ciamlogin.com or https://tenant_name.b2clogin.com/tenant.onmicrosoft.com/policy.  Also please double check your tenant name or GUID is correct.

📊 VALIDATION SUMMARY
✅ Valid: 2/11
❌ Invalid: 9/11
Success Rate: 18.2%

🔍 DETAILED RESULTS:
  OpenAI: ❌ INVALID
    └── Invalid - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-live-********************************************z4ZK. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
  Anthropic: ❌ INVALID
    └── Invalid - Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}
  Cohere: ❌ INVALID
    └── 